# Metadata

```yaml
course:   DS 5001 
topic:    Generating Data Files
author:   Andrew Chaphiv (acgq2@virginia.edu)
date:    SPR2023
```


# Importing Modules

In [35]:
import pandas as pd 
import numpy as np
import nltk
import re 

# Subsetting DataFrame

In [1]:
# There was a bunch of nonenglish articles present in the initial files, so need to get rid of those 
df = pd.read_csv("condensedabstracts.csv", index_col = 0)
non_english = df.Authors.str.match("\[(Article\s+in\s+[A-Za-z-]+)\]").to_frame() # Some articles are not in english, need to discard.
non_english = non_english[non_english["Authors"] == True].index
chinese = df.Authors.str.match("[Article in Chinese; Abstract available in Chinese from the publisher]").to_frame()
chinese = chinese[chinese["Authors"] == True].index
df = df.drop(chinese)
chinese = df.Authors.str.find("[Article in Chinese; Abstract available in Chinese from the publisher]").to_frame()
chinese = chinese[chinese["Authors"] != -1].index
df = df.drop(chinese)
df = df.drop(non_english)

df = df.reset_index(inplace=False, drop = True)
df.index.names = ["abstract_num"] 

df.to_csv("abstracts-LIB.csv")

NameError: name 'pd' is not defined

# Tokenizing

In [8]:
PARAS = df["Abstract"].to_frame('para_str')
PARAS

,para_str
abstract_num,
0,Interleukin-6 (IL-6) has previously been shown...
1,The goal of screening programmes for cancer is...
2,Non-parametric methods have recently been prop...
3,The observation that charcoal-treated fetal bo...
4,A new de-N-acetylated glycosphingolipid termed...
...,...
9793,OBJECTIVE: To characterize the supportive care...
9794,INTRODUCTION: Outcomes in colorectal cancer tr...
9795,OBJECTIVES: Chronic thromboembolic pulmonary h...


In [18]:
SENTS = PARAS.para_str.apply(lambda x: pd.Series(nltk.sent_tokenize(x)))\
        .stack()\
        .to_frame('sent_str')
SENTS.index.names = ["abstract_num", "sent_num"]

In [20]:
SENTS

sent_str
abstract_num sent_num                                                   
0            0         Interleukin-6 (IL-6) has previously been shown...
             1         However, the mechanisms leading to increased I...
             2         We have studied the effects of synthetic ceram...
             3         The synthetic ceramides C2- and C6-ceramide as...
             4         We propose that the sphingomyelin pathway is p...
...                                                                  ...
9797         10        Histopathology identified high-grade myxofibro...
             11        Postoperative intensity-modulated radiation th...
             12        The patient had greatly improved neurological ...
             13        CONCLUSIONS We reported a case of an unresecta...
             14        This combination therapy is a relatively safe ...

[82928 rows x 1 columns]

In [34]:
TOKENS = SENTS.sent_str\
            .apply(lambda x: pd.Series(nltk.pos_tag(nltk.WhitespaceTokenizer().tokenize(x))))\
            .stack()\
            .to_frame('pos_tuple')

In [36]:
TOKENS.index.names = ["abstract_num", "sent_num", "token_num"]
TOKENS['pos'] = TOKENS.pos_tuple.apply(lambda x: x[1])
TOKENS['token_str'] = TOKENS.pos_tuple.apply(lambda x: x[0])
TOKENS['term_str'] = TOKENS.token_str.str.lower()

In [37]:
TOKENS = TOKENS[TOKENS.term_str != '']
TOKENS

pos_tuple  pos      token_str  \
abstract_num sent_num token_num                                            
0            0        0          (Interleukin-6, JJ)   JJ  Interleukin-6   
                      1                 ((IL-6), NN)   NN         (IL-6)   
                      2                   (has, VBZ)  VBZ            has   
                      3             (previously, RB)   RB     previously   
                      4                  (been, VBN)  VBN           been   
...                                              ...  ...            ...   
9797         14       28                   (the, DT)   DT            the   
                      29                 (size,, NN)   NN          size,   
                      30             (location,, NN)   NN      location,   
                      31                    (or, CC)   CC             or   
                      32             (adhesion., NN)   NN      adhesion.   

                                      term_str  
abstract_num sent_num token_num                 
0            0        0          interleukin-6  
                      1                 (il-6)  
                      2                    has  
                      3             previously  
                      4                   been  
...                                        ...  
9797         14       28                   the  
                      29                 size,  
                      30             location,  
                      31                    or  
                      32             adhesion.  

[2013242 rows x 4 columns]

In [38]:
TOKENS.to_csv("abstracts-CORPUS.csv")

# Generating Vocab Table

In [39]:
def bag_words(df, bag = ["sent_num"]):
    bow = df.groupby(bag+['term_str']).term_str.count().to_frame('n')
    return bow

def TFIDF(BOW, tf_method = 'sum'):
    #sum, max, log, double_norm, raw, binary
    DTCM = BOW.n.unstack().fillna(0).astype('int')
    if tf_method == 'sum':
        TF = DTCM.T / DTCM.T.sum()
    elif tf_method == 'max':
        TF = DTCM.T / DTCM.T.max()
    elif tf_method == 'log':
        TF = np.log2(1 + DTCM.T)
    elif tf_method == 'raw':
        TF = DTCM.T
    elif tf_method == 'double_norm':
        TF = DTCM.T / DTCM.T.max()
    elif tf_method == 'binary':
        TF = DTCM.T.astype('bool').astype('int')
    TF = TF.T
    N = DTCM.shape[0]
    DF = DTCM.astype('bool').sum()
    IDF = np.log2(N / DF)
    TFIDF = TF * IDF
    BOW['tf'] = TF.stack()
    BOW['tfidf'] = TFIDF.stack()
    VOCAB['df'] = DF
    VOCAB['idf'] = IDF
    return TFIDF

In [40]:
VOCAB = TOKENS.term_str.value_counts().to_frame('n')
VOCAB.index.name = 'term_str'
VOCAB['p'] = VOCAB.n / VOCAB.n.sum()
VOCAB['i'] = -np.log2(VOCAB.p)
VOCAB['n_chars'] = VOCAB.index.str.len()
VOCAB['max_pos'] = TOKENS[['term_str','pos']].value_counts().unstack(fill_value=0).idxmax(1)
VOCAB['cat_pos'] = TOKENS[['term_str','pos']].value_counts().to_frame('n').reset_index()\
    .groupby('term_str').pos.apply(lambda x: set(x))
sw = pd.DataFrame(nltk.corpus.stopwords.words('english'), columns=['term_str'])
sw = sw.reset_index().set_index('term_str')
sw.columns = ['dummy']
sw.dummy = 1
VOCAB['stop'] = VOCAB.index.map(sw.dummy)
VOCAB['stop'] = VOCAB['stop'].fillna(0).astype('int')

from nltk.stem.porter import PorterStemmer
stemmer1 = PorterStemmer()
VOCAB['stem_porter'] = VOCAB.apply(lambda x: stemmer1.stem(x.name), 1)

from nltk.stem.snowball import SnowballStemmer
stemmer2 = SnowballStemmer("english")
VOCAB['stem_snowball'] = VOCAB.apply(lambda x: stemmer2.stem(x.name), 1)

from nltk.stem.lancaster import LancasterStemmer
stemmer3 = LancasterStemmer()
VOCAB['stem_lancaster'] = VOCAB.apply(lambda x: stemmer3.stem(x.name), 1)

In [41]:
bow = bag_words(TOKENS)
tfidf = TFIDF(bow, tf_method = 'max')
VOCAB['tfidf_mean'] = bow.groupby('term_str').tfidf.mean() #TFIDF[TFIDF > 0].mean().fillna(0) # EXPLAIN
VOCAB['tfidf_sum'] = tfidf.sum()
VOCAB['tfidf_median'] = bow.groupby('term_str').tfidf.median() #TFIDF[TFIDF > 0].median().fillna(0) # EXPLAIN
VOCAB['tfidf_max'] = tfidf.max()
VOCAB['dfidf'] = VOCAB.df * VOCAB.idf

In [42]:
VOCAB.to_csv("abstracts-VOCAB.csv")